# Percentage from ATL comparison

In this exercise, we will fetch top 250 cryptocurrencies by market cap from www.coingecko.com and compare their percentage change from their all time low price. This comparison data is good for figuring out what might be a good price to invest in for these cryptocurrencies.

# Library Imports

First we will import the list of open source libraries that we will need for this exercise.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

# Importing List of Top 250 cryptocurrencies

Next, we will import the list of Top 250 cryptocurrencies as on https://www.coingecko.com/.

The cryptocurrencies in this list may change over time so the ideal way to get the list of top 100 cryptocurrencies is to connect to the Coingecko API and get the real time data from their API database. The API documentation can be found here: https://www.coingecko.com/en/api/documentation.

Since this is a one time exercise, we will be using the free API version, there is no need to get an API key now. However, if we need to access the database frequently and choose a paid plan, an API key will be required. We will be using the /coins/markets API end point to get the list of top 250 cryptocurrencies. The end point also includes other relevant data as well, such as, ticker, price, market capitalization, atl, ath, etc which we will need later on.

Let's import the list to our Jupyter notebook using the following commands:

In [2]:
api_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page=1&sparkline=false"
data = requests.get(api_url).json()
print(data)

[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579', 'current_price': 38699, 'market_cap': 734817673028, 'market_cap_rank': 1, 'fully_diluted_valuation': 812852778386, 'total_volume': 19974946893, 'high_24h': 39715, 'low_24h': 38321, 'price_change_24h': -285.083270128176, 'price_change_percentage_24h': -0.73128, 'market_cap_change_24h': -5713359605.743652, 'market_cap_change_percentage_24h': -0.77152, 'circulating_supply': 18983968.0, 'total_supply': 21000000.0, 'max_supply': 21000000.0, 'ath': 69045, 'ath_change_percentage': -43.95478, 'ath_date': '2021-11-10T14:24:11.849Z', 'atl': 67.81, 'atl_change_percentage': 56966.60799, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2022-03-15T11:42:37.095Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880', 'current_price': 2538.1, 'market_cap

# Creating a Pandas DataFrame and parsing our API call

Next we will create a pandas dataframe where we can store the above data. We will call our dataframe "atl_dataframe".

In [3]:
my_columns = ['Name', 'Ticker','Current Price', 'Market Capitalization', 'ATH', 'ATH Date', 'ATH Change %', 'ATL', 'ATL Date', 'ATL Change %']
atl_dataframe = pd.DataFrame(columns = my_columns)
atl_dataframe

,Name,Ticker,Current Price,Market Capitalization,ATH,ATH Date,ATH Change %,ATL,ATL Date,ATL Change %


In [4]:
for i in data:
    name = i['name']
    ticker = i['symbol']
    price = i['current_price']
    market_cap = i['market_cap']
    ath = i['ath']
    ath_date = i['ath_date']
    ath_change = i['ath_change_percentage']
    atl = i['atl']
    atl_date = i['atl_date']
    atl_change = i['atl_change_percentage']
    atl_dataframe = atl_dataframe.append(
    pd.Series(
        [
            name,
            ticker,
            price,
            market_cap,
            ath,
            ath_date,
            ath_change,
            atl,
            atl_date,
            atl_change,
            
        ],
            index = my_columns,
    ),
        ignore_index = True
)
atl_dataframe

,Name,Ticker,Current Price,Market Capitalization,ATH,ATH Date,ATH Change %,ATL,ATL Date,ATL Change %
0,Bitcoin,btc,38699,734817673028,69045,2021-11-10T14:24:11.849Z,-43.95478,67.81,2013-07-06T00:00:00.000Z,56966.60799
1,Ethereum,eth,2538.1,304520231739,4878.26,2021-11-10T14:24:19.604Z,-47.97427,0.432979,2015-10-20T00:00:00.000Z,586060.44558
2,Tether,usdt,1.0,80074450088,1.32,2018-07-24T00:00:00.000Z,-24.41477,0.572521,2015-03-02T00:00:00.000Z,74.67738
3,BNB,bnb,365.78,61502493668,686.31,2021-05-10T07:24:17.097Z,-46.70779,0.039818,2017-10-19T00:00:00.000Z,918456.25818
4,USD Coin,usdc,1.0,52430740341,1.17,2019-05-08T00:40:28.300Z,-14.76805,0.891848,2021-05-19T13:14:05.611Z,12.07295
...,...,...,...,...,...,...,...,...,...,...
245,Energy Web Token,ewt,4.88,204794612,22.67,2021-04-17T03:10:00.960Z,-78.48165,0.543628,2020-04-03T00:00:00.000Z,797.46561
246,XYO Network,xyo,0.016387,204302752,0.081391,2021-11-06T04:44:55.346Z,-79.86676,0.000097,2020-03-13T02:24:24.303Z,16841.98756
247,Medibloc,med,0.03814,203807090,0.351852,2021-04-02T04:29:08.791Z,-89.18181,0.001614,2020-03-13T02:20:18.722Z,2257.82167
248,Astar,astr,0.099902,200394348,0.421574,2022-01-17T13:34:36.289Z,-76.22497,0.082846,2022-01-26T08:45:24.106Z,20.98249


# Output the result in an Excel file

Finally, we will output the above dataframe result in an excel file using xlsxwriter.

In [5]:
writer = pd.ExcelWriter('atl_comparison.xlsx', engine='xlsxwriter')
atl_dataframe.to_excel(writer, sheet_name='ATL Comparison', index = False)

In [6]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [7]:
column_formats = { 
                    'A': ['Name', string_template],
                    'B': ['Ticker', string_template],
                    'C': ['Current Price', dollar_template],
                    'D': ['Market Capitalization', dollar_template],
                    'E': ['ATH', dollar_template],
                    'F': ['ATH Date', string_template],
                    'G': ['ATH Change %', percent_template],
                    'H': ['ATL', dollar_template],
                    'I': ['ATL Date', string_template],
                    'J': ['ATL Change %', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['ATL Comparison'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['ATL Comparison'].write(f'{column}1', column_formats[column][0], string_template)

In [8]:
writer.save()